In [9]:
import db.knowhere_db as kdb
import pipeline.pipeline as pipeline
import pandas as pd
import numpy as np
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier

In [2]:
reader = kdb.Reader('knowhere')

In [3]:
glen_H_data_raw = reader.get_dataframe_pivoted(collection='iphone_test3', username='glen', min_date='2017-03-17', max_date='2017-03-20')

In [4]:
glen_H_data = glen_H_data_raw[['Acceleration x','Acceleration y','Acceleration z', 'Altimeter (Barometer) Pressure',\
                               'Microphone Left Channel Level', 'Microphone Right Channel Level','Magnetometer x',\
                               'Magnetometer y','Magnetometer z','Gyrometer x','Gyrometer y',\
                               'Gyrometer z']]
glen_H_data = glen_H_data.dropna().astype(float)

In [5]:
glen_H_data['Acceleration'] =  np.sqrt(glen_H_data['Acceleration x']**2 + glen_H_data['Acceleration y']**2 +\
                                       glen_H_data['Acceleration z']**2)
glen_H_data['Magnetometer'] =  np.sqrt(glen_H_data['Magnetometer x']**2 + glen_H_data['Magnetometer y']**2 +\
                                       glen_H_data['Magnetometer z']**2)
glen_H_data['Gyrometer']    =  np.sqrt(glen_H_data['Gyrometer x']**2 + glen_H_data['Gyrometer y']**2 +\
                                       glen_H_data['Gyrometer z']**2)
glen_H_data['Microphone']    =  (glen_H_data['Microphone Left Channel Level'] +\
                                glen_H_data['Microphone Right Channel Level'])/2
glen_H_data = glen_H_data[['Acceleration','Magnetometer','Gyrometer','Microphone','Altimeter (Barometer) Pressure']]

In [6]:
# set window
window = 5
# Rolling Means
glen_H_data['RollingMeanAcceleration'] = pd.rolling_mean(glen_H_data['Acceleration'], window)
glen_H_data['RollingMeanMagnetometer'] = pd.rolling_mean(glen_H_data['Magnetometer'], window)
glen_H_data['RollingMeanGyrometer'] = pd.rolling_mean(glen_H_data['Gyrometer'], window)
glen_H_data['RollingMeanMicrophone'] = pd.rolling_mean(glen_H_data['Microphone'], window)
glen_H_data['RollingMeanAltimeter'] = pd.rolling_mean(glen_H_data['Altimeter (Barometer) Pressure'], window)
# Rolling 75th percentile
glen_H_data['Rolling75thAcceleration'] = pd.rolling_quantile(glen_H_data['Acceleration'], window, 0.75)
glen_H_data['Rolling75thMagnetometer'] = pd.rolling_quantile(glen_H_data['Magnetometer'], window, 0.75)
glen_H_data['Rolling75thGyrometer'] = pd.rolling_quantile(glen_H_data['Gyrometer'], window, 0.75)
glen_H_data['Rolling75thMicrophone'] = pd.rolling_quantile(glen_H_data['Microphone'], window, 0.75)
glen_H_data['Rolling75thAltimeter'] = pd.rolling_quantile(glen_H_data['Altimeter (Barometer) Pressure'], window, 0.75)
# Rolling Max
glen_H_data['RollingMaxAcceleration'] = pd.rolling_max(glen_H_data['Acceleration'], window)
glen_H_data['RollingMaxMagnetometer'] = pd.rolling_max(glen_H_data['Magnetometer'], window)
glen_H_data['RollingMaxGyrometer'] = pd.rolling_max(glen_H_data['Gyrometer'], window)
glen_H_data['RollingMaxMicrophone'] = pd.rolling_max(glen_H_data['Microphone'], window)
glen_H_data['RollingMaxAltimeter'] = pd.rolling_max(glen_H_data['Altimeter (Barometer) Pressure'], window)
# Rolling Min
glen_H_data['RollingMinAcceleration'] = pd.rolling_min(glen_H_data['Acceleration'], window)
glen_H_data['RollingMinMagnetometer'] = pd.rolling_min(glen_H_data['Magnetometer'], window)
glen_H_data['RollingMinGyrometer'] = pd.rolling_min(glen_H_data['Gyrometer'], window)
glen_H_data['RollingMinMicrophone'] = pd.rolling_min(glen_H_data['Microphone'], window)
glen_H_data['RollingMinAltimeter'] = pd.rolling_min(glen_H_data['Altimeter (Barometer) Pressure'], window)
# Rolling st dev
glen_H_data['RollingSDAcceleration'] = pd.rolling_std(glen_H_data['Acceleration'], window)
glen_H_data['RollingSDMagnetometer'] = pd.rolling_std(glen_H_data['Magnetometer'], window)
glen_H_data['RollingSDGyrometer'] = pd.rolling_std(glen_H_data['Gyrometer'], window)
glen_H_data['RollingSDMicrophone'] = pd.rolling_std(glen_H_data['Microphone'], window)
glen_H_data['RollingSDAltimeter'] = pd.rolling_std(glen_H_data['Altimeter (Barometer) Pressure'], window)
# Rolling var
glen_H_data['RollingVarAcceleration'] = pd.rolling_var(glen_H_data['Acceleration'], window)
glen_H_data['RollingVarMagnetometer'] = pd.rolling_var(glen_H_data['Magnetometer'], window)
glen_H_data['RollingVarGyrometer'] = pd.rolling_var(glen_H_data['Gyrometer'], window)
glen_H_data['RollingVarMicrophone'] = pd.rolling_var(glen_H_data['Microphone'], window)
glen_H_data['RollingVarAltimeter'] = pd.rolling_var(glen_H_data['Altimeter (Barometer) Pressure'], window)

/Users/gafergus/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
/Users/gafergus/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:5: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
/Users/gafergus/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:6: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
/Users/gafergus/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:7: FutureWarning: pd.rolling_mean is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=5,center=False).mean()
/Users/gafergus/anaconda/lib/python2.7/site-packages/ipykern

In [7]:
glen_H_data = glen_H_data.dropna()

In [8]:
# load dataset
dataset = dataframe.values
X = dataset[:,0:35].astype(float)
Y = dataset[:,36]
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# One Hot Encode
lb = LabelBinarizer()
lb.fit(encoded_Y)
dummy_y = lb.fit_transform(encoded_Y)

In [ ]:
# build a classifier
RF_Class = RandomForestClassifier(n_estimators=100)
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [2, 3, None],
              "max_features": sp_randint(1,3),
              "min_samples_split": sp_randint(2, 3),
              "min_samples_leaf": sp_randint(1, 3),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(RF_Class, param_distributions=param_dist, n_iter=n_iter_search)

start = time()
random_search.fit(X, dummy_y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

In [ ]:
# logistic regression
logreg = sklearn.linear_model.ElasticNetCV(l1_ratio = [0, .25, .5, .75, 1.], cv=3)
logreg.fit(X, dummy_y)

In [ ]:
# build a classifier
ET_Class = ExtraTreesClassifier(n_estimators=100)
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")


# specify parameters and distributions to sample from
param_dist = {"max_depth": [2, 3, None],
              "max_features": sp_randint(1,3),
              "min_samples_split": sp_randint(2, 3),
              "min_samples_leaf": sp_randint(1, 3),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run randomized search
n_iter_search = 10
random_search = RandomizedSearchCV(ET_Class, param_distributions=param_dist, n_iter=n_iter_search)

start = time()
random_search.fit(X, dummy_y)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)

In [ ]:
# Do the Voting Classifier
VCM = VotingClassifier(estimators=[('RF_Class', RF_Class), ('logreg', logreg), ('ET_Class', ET_Class)], voting='hard')
VCM.fit(X, dummy_y)
VCM.fit_transform(X, dummy_y)